In [3]:
import sys
from pathlib import Path
sys.path.insert(0, str(Path.cwd().parent))
import pandas as pd
import numpy as np
seasonal = pd.read_parquet('../data/raw/seasonal_20_25.parquet')
pbp = pd.read_parquet('../data/raw/pbp_20_25.parquet')
roster = pd.read_parquet('../data/raw/seasonal_roster_20_25.parquet')
id = pd.read_parquet('../data/raw/ids.parquet')


In [ ]:
seasonal.columns.tolist()

In [ ]:
roster["position"].value_counts()
roster.columns.tolist()

In [ ]:
id.columns.tolist()

In [ ]:
roster.head()

In [ ]:
seasonal_ids = set(seasonal['player_id'])
roster_ids = set(roster['player_id'])


In [ ]:
len(seasonal_ids & roster_ids)  # how many IDs appear in both?


In [ ]:
seasonal['player_id'].duplicated().sum()



In [ ]:
seasonal['player_id'].nunique()  # unique players

In [ ]:
len(seasonal)  # total rows


In [ ]:
# In your notebook, test this:
ROSTER_KEEP_COLS = [
        "player_id",
        "season",
        "team",
        "position",
        "age",
        "height",
        "weight",
        "years_exp",
        "college",
        "rookie_year",
        "draft_number",
        "draft_club",
    ]
SEASON_KEEP_COLS = [
        # Keys
        "player_id", "season", "season_type", "games",

        # Passing
        "completions", "attempts", "passing_yards", "passing_tds",
        "interceptions", "sacks", "passing_first_downs", "passing_epa",

        # Rushing
        "carries", "rushing_yards", "rushing_tds",
        "rushing_first_downs", "rushing_epa",

        # Receiving
        "targets", "receptions", "receiving_yards", "receiving_tds",
        "receiving_first_downs", "receiving_epa",

        # Efficiency
        "pacr", "racr", "dakota", "yptmpa",

        # Opportunity / role
        "target_share", "air_yards_share", "wopr_x", "dom", "w8dom",

        # Fantasy outputs
        "fantasy_points", "fantasy_points_ppr", "ppr_sh"
    ]

filtered_roster = roster[roster["position"].isin(["QB", "RB", "WR", "TE"])]
season_filtered = seasonal[SEASON_KEEP_COLS]
end_roster = (
    filtered_roster[ROSTER_KEEP_COLS + ["week"]]
    .sort_values(["season", "week"])
    .groupby(["player_id", "season"], as_index=False)
    .last()
    .drop(columns=["week"], errors="ignore")
    )
assert end_roster.groupby(["player_id", "season"]).size().max() == 1
left_count = season_filtered.merge(end_roster, on=["player_id", "season"], how="left").shape[0]
inner_count = season_filtered.merge(end_roster, on=["player_id", "season"], how="inner").shape[0]

print(f"Left: {left_count}, Inner: {inner_count}, Dropped: {left_count - inner_count}")


In [4]:
from src.data.pipeline import load_data, clean_data
seasonal, roster = load_data()
df = clean_data(roster, seasonal)

df.isnull().sum()


ImportError: cannot import name 'NFLDataScraper' from 'src.data.scraper' (c:\Users\calde\Desktop\DraftBot\src\data\scraper.py)